Model Evaluation - Classification Template

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

from config import modelling_file_path, model_file_path
from src.modelling_data_contract import ModellingDataContract
from src.modelling.model_evaluation import XGBClassifierEvaluator

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model

In [ ]:
model_version = 1
model_name = ""

Response

In [ ]:
RESPONSE = ModellingDataContract.RESPONSE

Features

In [ ]:
FEATURES = ModellingDataContract.feature_list

Load Data

In [ ]:
modelling_data = pd.read_csv(modelling_file_path)
modelling_data.tail()

In [ ]:
training_data = modelling_data[modelling_data[RESPONSE+"TrainingSet"]]
test_data = modelling_data[modelling_data[RESPONSE+"TestSet"]]
cal_data = modelling_data[modelling_data[RESPONSE+"ValidationSet"]]

In [ ]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [ ]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]

Load Model

In [ ]:
super_xgb = joblib.load(model_file_path + "/" + model_name + "_v" + str(model_version) + ".joblib")

In [ ]:
super_xgb.xgb_model

Get Predictions

In [ ]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [ ]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

In [ ]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Evaluate Model

In [ ]:
training_data['xgb_preds'] = train_preds
training_data['xgb_probas'] = train_probas
training_data['xgb_cal_probas'] = train_cal_probas

train_xgb_evals = XGBClassifierEvaluator(model = super_xgb.xgb_model,
                                       data = training_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_cal_probas",
                                       expected_label_name = "xgb_preds"
                                       )

In [ ]:
test_data['xgb_preds'] = test_preds
test_data['xgb_probas'] = test_probas
test_data['xgb_cal_probas'] = test_cal_probas

test_xgb_evals = XGBClassifierEvaluator(model = super_xgb.xgb_model,
                                       data = test_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_cal_probas",
                                       expected_label_name = "xgb_preds"
                                       )

Model Averages

In [ ]:
training_data[[RESPONSE, 'xgb_probas', 'xgb_cal_probas']].describe()

In [ ]:
test_data[[RESPONSE, 'xgb_probas', 'xgb_cal_probas']].describe()

Plots

In [ ]:
train_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")
test_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")

In [ ]:
train_xgb_evals.plot_shap_summary_plot()
test_xgb_evals.plot_shap_summary_plot()

In [ ]:
top_features = test_xgb_evals.get_ranked_feature_importance()
top_10_features = top_features[:10]
top_10_features

In [ ]:
# for col in top_10_features:
#     train_xgb_evals.plot_pdp([col])
#     test_xgb_evals.plot_pdp([col])

In [ ]:
# for col in top_10_features:
#     train_xgb_evals.plot_ice([col])
#     test_xgb_evals.plot_ice([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_feature_ave(col)
    test_xgb_evals.plot_feature_ave(col)

Model Metrics (for probabilities, mainly want calibration curve, logloss and brier score loss)

In [ ]:
train_xgb_evals.get_log_loss(), test_xgb_evals.get_log_loss()

In [ ]:
train_xgb_evals.get_brier_score_loss(), test_xgb_evals.get_brier_score_loss()

In [ ]:
train_xgb_evals.display_calibration_curve(nbins=20)
test_xgb_evals.display_calibration_curve(nbins=20)